### Set up model and define necessary stuff

In [1]:
import torchvision
from torchvision.transforms import v2 as transforms
import torch
from models.centernet import ModelBuilder
from data.dataset import Dataset
from torch.utils.data import Subset
from postprocess_visual.object_detection_visualizer import (
    ObjectDetectionVisualizer,
)
from utils.load_model import load_model
from utils.predictions import get_predictions


Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def transform_dataset(dataset):
    """Transform the dataset for visualization"""
    transform = transforms.Compose(
        [
            transforms.Resize(size=(256, 256)),
        ]
    )

    return Dataset(dataset=dataset, transformation=transform)


model = load_model(Device, ModelBuilder, alpha=0.25)
model.eval()

loss_dict = {}


### Get predictions for defined dataset

In [ ]:
def prepare_dataset():
    # Load VOC dataset
    dataset_val = torchvision.datasets.VOCDetection(
        root="../VOC", year="2007", image_set="train", download=False
    )
    dataset_val = torchvision.datasets.wrap_dataset_for_transforms_v2(dataset_val)

    # Define a dataset that is a subset of the initial dataset
    indices = range(10)
    dataset_val = Subset(dataset_val, indices)

    return dataset_val


dataset = prepare_dataset()

# Transform the dataset to the correct form for further processing
dataset_transformed = transform_dataset(dataset)

# Get predictions
predictions = get_predictions(Device, model, dataset_transformed)

### Predictions visualization

In [ ]:
# Create visualizer with default settings
visualizer = ObjectDetectionVisualizer(
    dataset=dataset_transformed,
    input_height=256,
    input_width=256,
    confidence_threshold=0.3,
)

# Visualize predictions
visualizer.visualize_predictions(predictions)